In [15]:
import numpy as np
import random
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import alpaca_trade_api as tradeapi
from dotenv import load_dotenv
import os
from alpaca_trade_api.rest import REST, TimeFrame
import hvplot.pandas
from scipy.interpolate import interp1d
import models as model

import warnings
warnings.filterwarnings('ignore')

In [16]:
load_dotenv()

True

In [18]:
def fetch_stock_data(start_date, end_date, tickers, timeframe = '1Day'):
    # Grabbing all necessary alpaca information
    alpaca_api_key = os.getenv("ALPACA_API_KEY")
    alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
    api = REST(alpaca_api_key, alpaca_secret_key, base_url = 'https://paper-api.alpaca.markets')
    start_date_iso = pd.Timestamp(start_date, tz = 'America/New_York').isoformat()
    end_date_iso = pd.Timestamp(end_date, tz = 'America/New_York').isoformat()
    # Creating dataframe
    df_ticker = api.get_bars(tickers, TimeFrame.Day, start = start_date_iso, end = end_date_iso).df
    # Selecting only close price data for each stock, and concatenating them into one dataframe
    dfs = {}
    for ticker in tickers:
        ticker_df = df_ticker.loc[df_ticker.symbol == ticker].drop('symbol', axis = 1)
        ticker_df = ticker_df['close']
        dfs[ticker] = ticker_df
    new_df = pd.concat(dfs.values(), axis = 1, keys = tickers)
    # Resetting index to have only date information
    new_df.reset_index(inplace = True)
    new_df.timestamp = new_df.timestamp.dt.date
    new_df.set_index('timestamp', inplace = True)
    new_df.index = pd.to_datetime(new_df.index)
    return new_df

In [20]:
stock_list = ['NVDA']
df = fetch_stock_data('2018-01-12', '2024-03-14', stock_list)
df

NVDA
timestamp          
2018-01-12  222.759
2018-01-16  220.110
2018-01-17  224.720
2018-01-18  224.440
2018-01-19  230.110
...             ...
2024-03-08  875.280
2024-03-11  857.740
2024-03-12  919.130
2024-03-13  908.880
2024-03-14  879.440

[1552 rows x 1 columns]